# Text-Guided Editing of Images (Using CLIP and StyleGAN)

In [1]:
#@title Setup (may take a few minutes)
!git clone https://github.com/khalilacheche/StyleCLIP.git

import os
os.chdir(f'./StyleCLIP')

!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# downloads StyleGAN's weights and facial recognition network weights
ids = ['1EM87UquaoQmk17Q8d5kYIAHqu0dkYqdT', '1N0MZSqPRJpLfP4mFQCS14ikrVSe8vQlL']
for file_id in ids:
  downloaded = drive.CreateFile({'id':file_id})
  downloaded.FetchMetadata(fetch_all=True)
  downloaded.GetContentFile(downloaded.metadata['title'])

Cloning into 'StyleCLIP'...
remote: Enumerating objects: 885, done.
remote: Counting objects: 100% (267/267), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 885 (delta 169), reused 196 (delta 140), pack-reused 618
Receiving objects: 100% (885/885), 241.88 MiB | 20.84 MiB/s, done.
Resolving deltas: 100% (314/314), done.
Updating files: 100% (260/260), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-j0x2su8n
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-j0x2su8n
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Creat

In [81]:
experiment_type = 'edit' #@param ['edit', 'free_generation']

semantic_part = "hair" #@param ["mouth","skin","eyes","nose","ears","eye_brows","hat","hair","neck"]

description = 'A person with ### hair' #@param {type:"string"}

latent_path = None #@param {type:"string"}

optimization_steps = 40 #@param {type:"number"}

clip_lambda = 1 #@param {type:"number"}

l2_lambda = 0.004 #@param {type:"number"}

loc_lambda = 0.00001 #@param {type:"number"}

id_lambda = 0.005 #@param {type:"number"}

stylespace = False #@param {type:"boolean"}

create_video = False #@param {type:"boolean"}

export_segmentation_image = True #@param {type:"boolean"}


In [ ]:
colors = ['black', 'blue', 'red', 'blonde', 'gray', 'purple', 'brown']

In [82]:
use_seed = True #@param {type:"boolean"}

seed = 1 #@param {type: "number"}

In [83]:
#@title Additional Arguments
args = {
    "description": description,
    "ckpt": "stylegan2-ffhq-config-f.pt",
    "stylegan_size": 1024,
    "lr_rampup": 0.05,
    "lr": 0.1,
    "step": optimization_steps,
    "mode": experiment_type,
    "clip_lambda": clip_lambda,
    "l2_lambda": l2_lambda,
    "id_lambda": id_lambda,
    "loc_lambda": loc_lambda,
    'work_in_stylespace': stylespace,
    "latent_path": latent_path,
    "truncation": 0.7,
    "save_intermediate_image_every": 1 if create_video else 20,
    "results_dir": "results",
    "ir_se50_weights": "model_ir_se50.pth",
    "segmentation_model": "face_segmentation",
    "semantic_part":semantic_part,
    "export_segmentation_image": export_segmentation_image
}

In [88]:
if use_seed:
  import torch
  torch.manual_seed(seed)
from optimization.run_optimization import main
from argparse import Namespace
from criteria.clip_loss import CLIPLoss
import clip
from PIL import Image
import torch
import torchvision.transforms as transforms


clip_loss_evals = {}
for color in colors:  
  args['description'] = description.replace('###', color)
  result = main(Namespace(**args))
  img_path = 'results/00059.jpg'
  image = Image.open(img_path)
  transform = transforms.ToTensor()
  img_tensor = transform(image).unsqueeze(0).cuda()
  clip_loss = CLIPLoss(Namespace(**args))
  clip_losses = {}
  for eval_color in colors:
    desc = description.replace('###', eval_color)
    text_inputs = torch.cat([clip.tokenize(desc)]).cuda()
    c_loss = clip_loss(img_tensor, text_inputs)
    clip_losses[eval_color] = c_loss
  clip_loss_evals[color] = clip_losses

print(clip_loss_evals)

Loading ResNet ArcFace
Loading Segmentation Models


loss: 0.9204, loc_loss: 7124.3711;: 100%|██████████| 40/40 [00:33<00:00,  1.18it/s]
